In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
import yfinance as yf
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
import numpy as np
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from pmdarima.arima import auto_arima
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential, load_model
from keras import layers
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import warnings
warnings.filterwarnings("ignore", category=pd.core.common.SettingWithCopyWarning)

In [4]:
def HoltWinter(historical_data): 
    model = ExponentialSmoothing(historical_data, trend = "add",
                                  seasonal = "add", initialization_method = "legacy-heuristic", seasonal_periods = 12)
    model_fit = model.fit() #smoothing_trend = 0.2
    forecast = model_fit.forecast(steps=10)
    params = model_fit.params
    fitted_values = model_fit.fittedvalues
    forecast_df = pd.DataFrame(forecast, columns=["Predicted Price"])
    return forecast_df, params, fitted_values

In [5]:
data = yf.download("MDO.DE", period = "max")
data.drop(columns = ["Open", "High", "Low", "Volume", "Adj Close"], axis = 1, inplace = True)

[*********************100%***********************]  1 of 1 completed


In [6]:
end_zeitpunkte = ["2023-07-07"]
end_zeitpunkte_ts = [datetime.strptime(end_zeitpunkte[0], "%Y-%m-%d").date()]

#hier Zeitraum der historischen Daten anpassen:
time_horizont = 3
start_zeitpunkte_ts = [end_zeitpunkte_ts[0]-relativedelta(years=time_horizont)]

In [13]:
filtered_df = data.loc[start_zeitpunkte_ts[0]:end_zeitpunkte_ts[0]]
hist_data = filtered_df[: len(filtered_df)-10]
hist_data_for_function = hist_data.copy()
prog_data = filtered_df[len(filtered_df)-10 :]
index_list = prog_data.index.tolist()

forecast_df, params, fitted_values = HoltWinter(hist_data_for_function)

forecast_df.set_index(pd.Index(index_list), inplace=True)
result = pd.concat([prog_data, forecast_df], axis=1)

C:\Users\pasca\anaconda3\envs\ProjektSS23\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

C:\Users\pasca\anaconda3\envs\ProjektSS23\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

C:\Users\pasca\anaconda3\envs\ProjektSS23\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



In [18]:
hist_data

Date
2020-07-07    165.147026
2020-07-08    165.323847
2020-07-09    164.130614
2020-07-10    162.196899
2020-07-13    163.409694
                 ...    
2023-06-19    270.529644
2023-06-20    268.682376
2023-06-21    269.304915
2023-06-22    269.620155
2023-06-23    267.328579
Length: 755, dtype: float64

In [14]:
fig2 = px.line(template = "simple_white")
fig2.add_trace(go.Scatter(x = result.index.strftime("%Y-%m-%d"), y = result["Close"], mode = "lines", name = "Tatsächlicher Kurs", line_color = "red"))
fig2.add_trace(go.Scatter(x = result.index.strftime("%Y-%m-%d"), y = result["Predicted Price"], mode = "lines", name = "Prognose", line_color = "blue"))
fig2.update_layout(xaxis_title = "Datum", yaxis_title = "MC Donalds Kurspreis in EUR", xaxis= {"type": "category"}, legend=dict(x=0, y=1))
fig2.write_image("Prog_Holt.pdf")

In [36]:
fig3 = px.line(template = "simple_white")
fig3.add_trace(go.Scatter(x = hist_data.index[629:], y = hist_data["Close"][629:], mode = "lines", name = "Tatsächliche Daten", line_color = "red"))
fig3.add_trace(go.Scatter(x = hist_data.index[629:], y = fitted_values[629:], mode = "lines", name = "Prognostizierte Daten", line_color = "blue"))
fig3.update_layout(xaxis_title = "Datum", yaxis_title = "MC Donalds Kurspreis in EUR", legend=dict(x=0, y=1))
fig3.write_image("Train_Holt.pdf")

In [9]:
prog_dataa

,Close
Date,
2023-06-19,269.000000
2023-06-20,269.200012
2023-06-21,269.200012
2023-06-22,267.299988
2023-06-23,267.600006
2023-06-26,263.899994
2023-06-27,265.000000
2023-06-28,266.500000
2023-06-29,269.899994
